In [1]:
import pandas as pd
import numpy as np

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed')

In [3]:
def label(A,B):
    if A is True:
        return 0
    if B is True:
        return 1
    return 2

In [4]:
train_data = gap_train.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
train_data.A_vector = train_data.A_vector.map(lambda x:np.mean(x,axis = 0))
train_data.B_vector = train_data.B_vector.map(lambda x:np.mean(x,axis = 0))
train_data.pron_vector = train_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
train_data["product_vector"] = train_data.A_vector*train_data.pron_vector
train_data["label"] = train_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
train_data = train_data.drop(columns= ["A-coref","B-coref"])

In [5]:
test_data = gap_test.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
test_data.A_vector = test_data.A_vector.map(lambda x:np.mean(x,axis = 0))
test_data.B_vector = test_data.B_vector.map(lambda x:np.mean(x,axis = 0))
test_data.pron_vector = test_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
test_data["product_vector"] = test_data.A_vector*test_data.pron_vector
#test_data["label"] = test_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
#test_data = test_data.drop(columns= ["A-coref","B-coref"])

In [6]:
valid_data = gap_valid.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
valid_data.A_vector = valid_data.A_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.B_vector = valid_data.B_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.pron_vector = valid_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
valid_data["product_vector"] = valid_data.A_vector*valid_data.pron_vector
valid_data["label"] = valid_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
valid_data = valid_data.drop(columns= ["A-coref","B-coref"])

In [12]:
columns = train_data.columns[:-1]
X_train = np.concatenate([np.array(list(train_data[col])).reshape(train_data.shape[0],-1) for col in columns],axis = 1)
y_train = list(train_data.label)   

In [13]:
X_valid = np.concatenate([np.array(list(valid_data[col])).reshape(valid_data.shape[0],-1) for col in columns],axis = 1)
y_valid = list(valid_data.label)   

In [14]:
X_test = np.concatenate([np.array(list(test_data[col])).reshape(test_data.shape[0],-1) for col in columns],axis = 1)

In [15]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_train, y_train)


/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [27]:
import pickle
f = open( "./temp_result/base_model_data", "wb" )
pickle.dump(X_train,  f)
pickle.dump(y_train,  f)
pickle.dump(X_valid,  f)
pickle.dump(y_valid,  f)
pickle.dump(X_test,  f)
f.close()

In [19]:
pred = clf.predict_proba(X_test)

In [21]:
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred

In [22]:
sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.795739,0.027751,0.176510
1,development-2,0.958475,0.039947,0.001579
2,development-3,0.085302,0.903700,0.010998
3,development-4,0.004008,0.875409,0.120583
4,development-5,0.000022,0.996505,0.003473


In [24]:
import datetime
sub_df.to_csv("./test_and_submit/submission@"+str(datetime.datetime.now())+".csv", index=False)